# 我要What系列  WeChaty + PaddleHub

## 一、项目背景
基于PaddleHub和PaddleGAN, 使用微信机器人学习体验一些图像和视频的处理。所使用的项目主要参考AISTUDIO上各位大佬开源的，我只是简单集成了, 可以帮助像我这样刚入门的新手畅游在PaddlePaddle的神奇魅力中。

本项目是参加【**AI ChatBot 创意赛**】的作品，PaddleHub携手开源聊天机器人框架WeChaty带来 AI ChatBot创意赛，为AI算法工程师提供一个全新的应用场景：Chatbot （Conversational AI），同时也为Chatbot 开发者提供一个全新的AI能力平台，拓宽视野，为未来设计更加强大的 Chatbot 提供一扇门。

本项目主要难点是不仅使用到了PaddleGAN的模型，还使用了PaddleHub里的开源模型，需要保证在同一系统下的兼容问题, 以及各个逻辑间协调问题， 增加了后续模型的可扩展性。本项目可供用户体验到AI广泛的趣味性

关于Wechaty的安装和使用请参考我另一篇： [微信医聊自动问答 WeChaty + PaddleHub ](https://aistudio.baidu.com/aistudio/projectdetail/1868162)

<iframe style="width:98%;height: 450px;" src="//player.bilibili.com/player.html?aid=757910829&bvid=BV1N64y127bE&cid=334876358&page=1" scrolling="no" border="0" frameborder="no" framespacing="0" allowfullscreen="true"> </iframe>


**b站地址:[https://www.bilibili.com/video/BV1N64y127bE/](https://www.bilibili.com/video/BV1N64y127bE/)**

## 二、安装python的依赖包

下载并安装PaddleGAN


```
!git clone https://gitee.com/paddlepaddle/PaddleGAN
!pip install -r PaddleGAN/requirements.txt imageio-ffmpeg
```



In [1]:
!pip install -r PaddleGAN/requirements.txt imageio-ffmpeg moviepy cmake boost dlib

## 三、我要唱蚂蚁呀嘿

First order motion model的任务是image animation，给定一张源图片，给定一个驱动视频，生成一段视频，其中主角是源图片，动作是驱动视频中的动作，源图像通常包含一个主体，驱动视频包含一系列动作。

以人脸表情迁移为例，给定一个源人物，给定一个驱动视频，可以生成一个视频，其中主体是源人物，视频中源人物的表情是由驱动视频中的表情所确定的。通常情况下，我们需要对源人物进行人脸关键点标注、进行表情迁移的模型训练。

但是这篇文章提出的方法只需要在同类别物体的数据集上进行训练即可，比如实现太极动作迁移就用太极视频数据集进行训练，想要达到表情迁移的效果就使用人脸视频数据集voxceleb进行训练。训练好后，我们使用对应的预训练模型就可以达到前言中实时image animation的操作。

```
def create_mayiyahe(img_path):
    os.system("export PYTHONPATH=$PYTHONPATH:/home/aistudio/PaddleGAN && python -u PaddleGAN/applications/tools/first-order-demo.py  --driving_video ~/work/fullbody.MP4  --source_image " + img_path+ "  --relative --adapt_scale")
    videoclip_1 = VideoFileClip("/home/aistudio/work/fullbody.MP4")
    videoclip_2 = VideoFileClip("/home/aistudio/output/result.mp4")
    audio_1 = videoclip_1.audio
    videoclip_3 = videoclip_2.set_audio(audio_1)
    videoclip_3.write_videofile("mayiyahei.mp4", audio_codec="aac")
```

**将驱动视频传至代码中，具体的各参数使用说明如下**

* 1. driving_video: 驱动视频，视频中人物的表情动作作为待迁移的对象
* 1. source_image: 原始图片，视频中人物的表情动作将迁移到该原始图片中的人物上
* 1. relative: 指示程序中使用视频和图片中人物关键点的相对坐标还是绝对坐标，建议使用相对坐标，若使用绝对坐标，会导致迁移后人物扭曲变形
* 1. adapt_scale: 根据关键点凸包自适应运动尺度
* 1. output：设置输出视频的存放文件夹


In [26]:
!python final_script.py 我要唱蚂蚁呀嘿 /home/aistudio/work/liudehua.jpg

我要唱蚂蚁呀嘿
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
[05/04 21:15:12] ppgan INFO: Found /home/aistudio/.cache/ppgan/vox-cpk.pdparams
W0504 21:15:12.439229  4345 device_context.cc:362] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0504 21:15:12.444278  4345 device_context.cc:372] device: 0, cuDNN Version: 7.6.
100%|████████████████████████

## 四、我要唱吻别

Wav2lip实现唇形与语音精准同步突破的关键在于，它采用了唇形同步判别器，以强制生成器持续产生准确而逼真的唇部运动。

此外，该研究通过在鉴别器中，使用多个连续帧而不是单个帧，并使用视觉质量损失（而不仅仅是对比损失）来考虑时间相关性，从而改善了视觉质量。

该wav2lip模型几乎是万能的，适用于任何人脸、任何语音、任何语言，对任意视频都能达到很高的准确率，可以无缝地与原始视频融合，还可以用于转换动画人脸，并且导入合成语音也是可行的

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/adb83d8b854246e0b713df86fa6fb6d4a714a01f01bb42d8a5ef50744ca7c13a" width='800' height=''></center>
<center><br>Wav2Lip唇形同步示意图</br></center>

使用一个强有力的嘴唇同步鉴别器迫使发生器产生精确的嘴唇形状。然而，它有时会导致变形区域稍微模糊或包含轻微的伪像。为了减轻这种微小的质量损失，我们在 GAN 设置中训练了一个简单的视觉质量鉴别器和生成器。因此，我们有两个鉴别器，一个用于同步精度，另一个用于更好的视觉质量。一方面，唇同步鉴别器不需要进一步微调，即训练过程权重被冻结。另一方面，由于视觉质量鉴别器不对嘴唇同步进行任何检查，并且只惩罚不现实的人脸生成，所以它是在生成的人脸上进行训练的。

Wav2Lip模型结构如下图所示：

<center><img src='https://ai-studio-static-online.cdn.bcebos.com/3fbd93a0d6c04b66bfd1ac2f958635e80d0c7cc883e54a9c888e867d564407af' width='800' height=''></center>
 

具体的参数使用说明如下：
* face: 原始视频，视频中的人物的唇形将根据音频进行唇形合成--通俗来说，想让谁说话
* audio：驱动唇形合成的音频，视频中的人物将根据此音频进行唇形合成--通俗来说，想让这个人说什么

```
def create_kissbye(img_path):
    os.system("export PYTHONPATH=$PYTHONPATH:/home/aistudio/PaddleGAN && python PaddleGAN/applications/tools/wav2lip.py --face " 
  + img_path + " --audio /home/aistudio/work/clip.wav --outfile wav2lip.mp4")
```


In [3]:
!python final_script.py 我要唱吻别 /home/aistudio/work/liudehua.jpg

2021-05-06 12:05:30,833 - WARNING - /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):

2021-05-06 12:05:31,993 - WARNING - /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pip/_vendor/packaging/version.py:130: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  DeprecationWarning,

[WARNING 2021-05-06 12:05:31,993 warnings.p

## 五、我要像她一样漂亮

[PSGAN](https://arxiv.org/abs/1909.06956)模型的任务是妆容迁移， 即将任意参照图像上的妆容迁移到不带妆容的源图像上。很多人像美化应用都需要这种技术。近来的一些妆容迁移方法大都基于生成对抗网络（GAN）。它们通常采用 CycleGAN 的框架，并在两个数据集上进行训练，即无妆容图像和有妆容图像。但是，现有的方法存在一个局限性：只在正面人脸图像上表现良好，没有为处理源图像和参照图像之间的姿态和表情差异专门设计模块。PSGAN是一种全新的姿态稳健可感知空间的生成对抗网络。PSGAN 主要分为三部分：妆容提炼网络（MDNet）、注意式妆容变形（AMM）模块和卸妆-再化妆网络（DRNet）。这三种新提出的模块能让 PSGAN 具备上述的完美妆容迁移模型所应具备的能力。


<div align="center">
  <img src="https://ai-studio-static-online.cdn.bcebos.com/1586183652c44b088fa0e7ae2215e67b843d41eb26154cbbba12628ddcdc2924" width="800"/>
</div>



```
def create_markup(img_path):
    os.system("python PaddleGAN/tools/psgan_infer.py --config-file PaddleGAN/configs/makeup.yaml --model_path /home/aistudio/work/psgan_weight.pdparams --source_path " + img_path + " --reference_dir PaddleGAN/docs/imgs/ref --evaluate-only")
```
**参数说明:**

* config-file: PSGAN网络到参数配置文件，格式为yaml
* model_path: 训练完成保存下来网络权重文件的路径
* source_path: 未化妆的原始图片文件全路径，包含图片文件名字
* reference_dir: 化妆的参考图片文件路径，不包含图片文件名字

In [34]:
!python final_script.py 我要像她一样漂亮 /home/aistudio/work/liudehua.jpg

我要像她一样漂亮
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
[05/04 22:02:48] ppgan INFO: Configs: {'epochs': 100, 'output_dir': 'tmp/makeup-2021-05-04-22-02', 'checkpoints_dir': 'checkpoints', 'model': {'name': 'MakeupModel', 'generator': {'name': 'GeneratorPSGANAttention', 'conv_dim': 64, 'repeat_num': 6}, 'discriminator': {'name': 'NLayerDiscriminator', 'ndf': 64, 'n_layers': 3, 'input_nc': 3, 'norm_t

## 六、我要变卡通

U-GAT-IT: Unsupervised Generative Attentional Networks with Adaptive Layer-Instance Normalization for Image-to-Image Translation
UGATIT是一种新的无监督图像到图像转换方法
UGATIT能够实现将真实人像图像转换成动漫人脸图像或将动漫人脸图像转换成真实人像图像

```
def create_cartoon(img_path):
    result = ugatit.style_transfer(
        images=None,
        paths=[img_path],
        batch_size=1,
        output_dir='output',
        visualization=True
    )
```

**参数说明：**
* images (list[numpy.ndarray]): 图片数据，ndarray.shape 为 [H, W, C]，默认为 None
* paths (list[str]): 图片的路径，默认为 None
* batch_size (int): batch 的大小，默认设为 1
* output_dir (str): 图片的保存路径，默认设为 output
* visualization (bool): 是否将识别结果保存为图片文件，默认设为 False

In [40]:
!python final_script.py 我要变卡通 /home/aistudio/work/liudehua.jpg

2021-05-04 22:16:06,429 - WARNING - /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):

2021-05-04 22:16:07,515 - WARNING - /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pip/_vendor/packaging/version.py:130: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  DeprecationWarning,

[WARNING 2021-05-04 22:16:07,515 warnings.p

## 七、我要变女生/金发/老

数据集使用的是CelebA（CelebFaces Attribute），名人人脸属性数据集，其包含10,177个名人身份的202,599张人脸图片，我用了后202,479张。每张图像的高×宽为218×178。
论文链接：https://arxiv.org/abs/1912.01865
论文视频：https://youtu.be/0EVh5Ki4dIY

StarGANv2由四个模块组成：

**生成器G**将输入图像转换为反映特定域风格编码的输出图像；

**映射网络F**将潜在代码转换为训练中随机选择的多个域的风格编码；

**风格编码器E**提取图像的风格编码，允许生成器执行参照物引导的图像合成；

**鉴别器D**从多个域中区分真实图像和虚假图像。

![](https://ai-studio-static-online.cdn.bcebos.com/9762a2450f334b83a98c752566e5bcfd33af18cfe5564f74842263b7a8141727)

```
#我要变女生
def create_female(img_path):
    input_dict = {"image": [img_path], "style": ["Female"]}
    results = stargan.generate(data=input_dict)
    print(results)

#我要变金发
def create_blondhair(img_path):
    input_dict = {"image": [img_path], "style": ["Blond_Hair"]}
    results = stargan.generate(data=input_dict)
    print(results)

#我要变老
def create_old(img_path):
    input_dict = {"image": [img_path], "style": ["Aged"]}
    results = stargan.generate(data=input_dict)
    print(results)
```

In [44]:
# !python final_script.py 我要变女生 /home/aistudio/work/liudehua.jpg
# !python final_script.py 我要变金发 /home/aistudio/work/liudehua.jpg
!python final_script.py 我要变老 /home/aistudio/work/liudehua.jpg

2021-05-04 22:30:03,413 - WARNING - /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):

2021-05-04 22:30:04,552 - WARNING - /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pip/_vendor/packaging/version.py:130: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  DeprecationWarning,

[WARNING 2021-05-04 22:30:04,552 warnings.p

## 八、我要去北京/柏林

通过加载PaddleHub DeepLabv3+模型(deeplabv3p_xception65_humanseg)实现一键抠图， 作者通过encoder-decoder进行多尺度信息的融合，同时保留了原来的空洞卷积和ASSP层， 其骨干网络使用了Xception模型，提高了语义分割的健壮性和运行速率，在 PASCAL VOC 2012 dataset取得新的state-of-art performance，该PaddleHub Module使用百度自建数据集进行训练，可用于人像分割，支持任意大小的图片输入。

```
#我要去柏林
def create_berlin(img_path):
    img = cv2.imread(img_path)
    results = humanseg.segmentation(images=[img], use_gpu=True, visualization=True)
    for result in results:
        blend_images(result["save_path"], "/home/aistudio/work/berlin.jpg")

#我要去北京
def create_beijing(img_path):
    img = cv2.imread(img_path)
    results = humanseg.segmentation(images=[img], use_gpu=True, visualization=True)
    for result in results:
        blend_images(result["save_path"], "/home/aistudio/work/beijing.jpg")
```


将抠出的人物图片合成在想要的背景图片当中。

```
def blend_images(fore_image, base_image):
    base_image = Image.open(base_image).convert('RGB')
    fore_image = Image.open(fore_image).resize(base_image.size)
    scope_map = np.array(fore_image)[:,:,-1] / 255
    scope_map = scope_map[:,:,np.newaxis]
    scope_map = np.repeat(scope_map, repeats=3, axis=2)
    res_image = np.multiply(scope_map, np.array(fore_image)[:,:,:3]) + np.multiply((1-scope_map), np.array(base_image))
    res_image = Image.fromarray(np.uint8(res_image))
    res_image.save("blend_res_img.jpg")
```

In [64]:
# !python final_script.py 我要去柏林 /home/aistudio/work/liudehua2.jpg
!python final_script.py 我要去北京 /home/aistudio/work/liudehua2.jpg

2021-05-04 23:26:16,118 - WARNING - /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):

2021-05-04 23:26:17,302 - WARNING - /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pip/_vendor/packaging/version.py:130: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  DeprecationWarning,

[WARNING 2021-05-04 23:26:17,302 warnings.p

## 九、部署在wechaty上

这里使用docker脚本可方便快速部署。请将your_token处替换成你的WeChaty token (**puppet_padlocal_xxxxxxxxxxxxx)**

```
export WECHATY_LOG="verbose"
export WECHATY_PUPPET="wechaty-puppet-padlocal"
export WECHATY_PUPPET_PADLOCAL_TOKEN="your_token"

export WECHATY_PUPPET_SERVER_PORT="8080"
export WECHATY_TOKEN="your_token"

docker run -ti \
  --name wechaty_puppet_service_token_gateway \
  --rm \
  -e WECHATY_LOG \
  -e WECHATY_PUPPET \
  -e WECHATY_PUPPET_PADLOCAL_TOKEN \
  -e WECHATY_PUPPET_SERVER_PORT \
  -e WECHATY_TOKEN \
  -p "$WECHATY_PUPPET_SERVER_PORT:$WECHATY_PUPPET_SERVER_PORT" \
  wechaty/wechaty:latest
~                        
```

运行成功后如下图:


![](https://ai-studio-static-online.cdn.bcebos.com/73f123bb9b37410eb49f834dc512226e014f3f53a77d42888b201833e8ef6942)

**实际效果展示:**

![](https://ai-studio-static-online.cdn.bcebos.com/fdc9dcabc7934bfd8bc706feb1ae3ab420fe54c3121b46c28ce8d7588d1aab0d)

![](https://ai-studio-static-online.cdn.bcebos.com/3d49d29d1055424d80315f7c9dba8f67cff6e3cf9e0f45d79c783db0bbcf8c29)

![](https://ai-studio-static-online.cdn.bcebos.com/f3f1706c74fa404b89cdef82c6caefa451435f0b13cd4fa5a28d129f8e630a6a)

![](https://ai-studio-static-online.cdn.bcebos.com/6893a9e8aaed450589aafce2f2ae11f8cf3106e804b44b6c9b627ff2d00fb473)


## 十、总结

后面计划若发现有意思的PaddleHub，将持续集成在这里，作为新手入门手册

# 最后，请各位朋友们给个一键三连（喜欢♡、fork〧、关注+）吧，助我上黄金，谢谢朋友们的支持！

**关于作者**

PaddlePaddle开发爱好者

我在AI Studio上获得黄金等级，点亮5个徽章，来互关呀~ https://aistudio.baidu.com/aistudio/personalcenter/thirdview/89442